In [1]:
import numpy as np
import json
import os
import itertools

In [2]:
# https://github.com/facebookresearch/mobile-vision/blob/master/runtime_lut/code/lut_schema.py

class LUTSchema:
    """
    Immutable dictionary that defines the LUT schema.
    """

    def __init__(self):
        self.record = {}
        # operator type
        self.record["op_type"] = ""
        # a list of argument definitions, sorted by argument names. Each argument is
        # represented by its proto string,  containing argument name and values.
        self.record["op_args"] = []
        # a list of input shapes. E.g., a convolution op may have shapes like:
        #   [[1, 3, 224, 224], [16, 3, 3, 3], [16]]
        # which are input tensor shape, kernel shape, and bias shape
        self.record["input_shapes"] = []
        # a list of integers indicating input data types
        self.record["input_dtypes"] = []
        # runtime measurement in micro-second, percentile-0
        self.record["runtime_us_p0"] = 0.0
        # runtime measurement in micro-second, percentile-10
        self.record["runtime_us_p10"] = 0.0
        # runtime measurement in micro-second, percentile-50
        self.record["runtime_us_p50"] = 0.0
        # runtime measurement in micro-second, percentile-90
        self.record["runtime_us_p90"] = 0.0
        # runtime measurement in micro-second, percentile-100
        self.record["runtime_us_p100"] = 0.0
        # benchmarking device
        self.record["device"] = ""

    def get_val(self, key):
        assert key in self.record, "%s not in schema" % key
        return self.record[key]

    def set_val(self, key, val):
        assert key in self.record, "%s not in schema" % key
        self.record[key] = val

    def get_dict_record(self):
        return self.record

    def load_from_json(self, input_):
        assert isinstance(input_, str) or isinstance(input_, dict)
        if isinstance(input_, str):
            record = json.loads(input_)
        elif isinstance(input_, dict):
            record = input_
        for k, v in record.items():
            if k == "op_args":
                v = [str(a) for a in v if "ws_nbytes_limit" not in str(a)]
            self.set_val(k, v)

## 1. Load LUT Database

In [19]:
# LUT Database Path
json_path = './Quantization/SM-G950U-7.0-24-op_lut_database_caffe2.json/SM-G950U-7.0-24-op_lut_database.json'

assert os.path.exists(json_path)

db_file = open(json_path, "r")
db = db_file.readlines()

# operation list
ops = []
for record in db:
    op = LUTSchema()
    op.load_from_json(json.loads(record))
    ops.append(op)

In [20]:
# check value list

op_type_list = [x.get_val('op_type') for x in ops]
op_type_list = list(set(op_type_list))

op_args_list = [x.get_val('op_args') for x in ops]
op_args_list.sort()
op_args_list = list(op_args_list for op_args_list,_ in itertools.groupby(op_args_list))

input_dtypes_list = [x.get_val('input_dtypes') for x in ops]
input_dtypes_list.sort()
input_dtypes_list = list(input_dtypes_list for input_dtypes_list,_ in itertools.groupby(input_dtypes_list))


In [21]:
op_type_list

['Int8ConvRelu',
 'GivenTensorIntFill',
 'Int8Softmax',
 'Int8Dequantize',
 'Int8Sum',
 'Int8MaxPool',
 'NHWC2NCHW',
 'Int8Conv',
 'Cast',
 'ConstantFill',
 'Int8ChannelShuffle',
 'Int8FC',
 'Int8Relu',
 'NCHW2NHWC',
 'MaxPool',
 'SpatialBN',
 'Conv',
 'GivenTensorInt64Fill',
 'ConvTranspose',
 'Transpose',
 'Relu',
 'Squeeze',
 'Shape',
 'AveragePool',
 'Softmax',
 'PadImage',
 'FC',
 'Int8AveragePool',
 'Add',
 'Int8Quantize',
 'ChannelShuffle',
 'Concat',
 'Sum']

In [22]:
ops[0].get_dict_record()

{'op_type': 'Int8Conv',
 'op_args': ['name: "exhaustive_search"\ni: 1\n',
  'name: "kernel"\ni: 1\n',
  'name: "order"\ns: "NHWC"\n',
  'name: "pad"\ni: 0\n',
  'name: "stride"\ni: 1\n'],
 'input_shapes': [[1, 7, 7, 306], [1228, 1, 1, 306], [1228]],
 'input_dtypes': [6, 6, 2],
 'runtime_us_p0': 648.0,
 'runtime_us_p10': 652.0,
 'runtime_us_p50': 923.0,
 'runtime_us_p90': 2540.0,
 'runtime_us_p100': 3382.0,
 'device': 'SM-G950U-7.0-24'}

In [150]:
{"runtime_us_p50": 874.0, "input_dtypes": [6, 6, 2], "runtime_us_p10": 850.0, "input_shapes": [[1, 224, 224, 3], [32, 3, 3, 3], [32]], "runtime_us_p0": 847.0, "op_args": ["name: \"exhaustive_search\"\ni: 1\n", "name: \"kernel\"\ni: 3\n", "name: \"order\"\ns: \"NHWC\"\n", "name: \"pad\"\ni: 1\n", "name: \"stride\"\ni: 2\n", "name: \"ws_nbytes_limit\"\ni: 268435456\n"], "runtime_us_p100": 1081.0, "op_type": "Int8Conv", "device": "SM-G950U-7.0-24", "runtime_us_p90": 924.0}

{'runtime_us_p50': 874.0,
 'input_dtypes': [6, 6, 2],
 'runtime_us_p10': 850.0,
 'input_shapes': [[1, 224, 224, 3], [32, 3, 3, 3], [32]],
 'runtime_us_p0': 847.0,
 'op_args': ['name: "exhaustive_search"\ni: 1\n',
  'name: "kernel"\ni: 3\n',
  'name: "order"\ns: "NHWC"\n',
  'name: "pad"\ni: 1\n',
  'name: "stride"\ni: 2\n',
  'name: "ws_nbytes_limit"\ni: 268435456\n'],
 'runtime_us_p100': 1081.0,
 'op_type': 'Int8Conv',
 'device': 'SM-G950U-7.0-24',
 'runtime_us_p90': 924.0}

## 2. Matching Search Space & LUT

In [46]:
model = dict()

In [47]:
search_space = ['k3_e6_g1', 'k3_e3_g1', 'k3_e1_g1', 'k3_e1_g2',
                'k5_e6_g1', 'k5_e3_g1', 'k5_e1_g1', 'k5_e1_g2', 'skip']
'''
[0,
== Search Layer START ==
1,
2, 3, 4, 5,
6, 7, 8, 9,
10, 11, 12, 13,
14, 15, 16, 17,
18, 19, 20, 21,
22, 
== Search Layer END ==
23, 24, 25]'''

stride2_layer = [0 ,2 ,6 ,10 ,18]
non_inverse_residual_block = [0, 2, 6, 10, 14, 18, 22, 23, 24, 25]

# 16 * ( 5 * 2 + 4 * 6) = 544
# 6 * ( 4 * 2 + 3 * 6) = 156
# => 700

In [48]:
# NHWC
input_shape = [[1, 224, 224, 3],
               [1, 112, 112, 16],
               [1, 112, 112, 16], [1, 56, 56, 24], [1, 56, 56, 24], [1, 56, 56, 24],
               [1, 56, 56, 24], [1, 28, 28, 32], [1, 28, 28, 32], [1, 28, 28, 32],
               [1, 28, 28, 32], [1, 14, 14, 64], [1, 14, 14, 64], [1, 14, 14, 64],
               [1, 14, 14, 64], [1, 14, 14, 112], [1, 14, 14, 112], [1, 14, 14, 112],
               [1, 14, 14, 112], [1, 7, 7, 184], [1, 7, 7, 184], [1, 7, 7, 184],
               [1, 7, 7, 184],
               [1, 7, 7, 352], [1, 7, 7, 1504], [1, 1, 1, 1504], [1000]]

In [49]:
# You can check input_shape form using get_runtime.py
# https://github.com/facebookresearch/mobile-vision/tree/master/runtime_lut

# To Be Searched Block (1 ~ 22)
for i in range(1, 23):
    layer = 'L' + str(i)
    model[layer] = dict()
    for search_candidate in search_space:
        if search_candidate == 'skip':
            model[layer][search_candidate] = list()
            continue
        else :
            split_parameter = search_candidate.split('_')
            k = int(split_parameter[0][1])
            e = int(split_parameter[1][1])
            g = int(split_parameter[2][1])
        
        model[layer][search_candidate] = dict()
        
        
        expansion_filter = input_shape[i][3] * e
        output_filter = input_shape[i+1][3]
        
        # depthwise separable conv - stride
        if i in stride2_layer:
            stride = 2
        else:
            stride = 1
        
        # find operation with [input_shape, operation, stride]
        # [optnal] + 'exhaustive_search' : overlapping operation exist.
        if g == 1:
            # Conv_Relu_0 (pointwise)
            model[layer][search_candidate]['op_1'] = {"input_shape" : [input_shape[i], [expansion_filter, 1, 1, input_shape[i][3]], [expansion_filter]],
                                                            "operation" : 'Int8ConvRelu',
                                                            "stride" : 1}          
            model[layer][search_candidate]['op_1']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_1']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_1']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_1']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])]
            # Conv_Relu_1 (depthwise)
            model[layer][search_candidate]['op_2'] = {"input_shape" : [input_shape[i][:3] + [expansion_filter], [expansion_filter, k, k, 1],[expansion_filter]],
                                                            "operation" : 'Int8ConvRelu',
                                                            "stride" : stride}
            model[layer][search_candidate]['op_2']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_2']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_2']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_2']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])]
            # Conv_Relu_2 (pointwise)                                                  
            model[layer][search_candidate]['op_3'] = {"input_shape" : [input_shape[i+1][:3] + [expansion_filter], [output_filter, 1, 1, expansion_filter],[output_filter]],
                                                            "operation" : 'Int8Conv',
                                                            "stride" : 1}
            model[layer][search_candidate]['op_3']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_3']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_3']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_3']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])] # and
                                                                           # 'exhaustive_search' not in x.get_dict_record()['op_args'][0]]
            # Int8Sum 
            if not i in non_inverse_residual_block:
                model[layer][search_candidate]['op_4'] = {"input_shape" : [input_shape[i+1], input_shape[i+1]],
                                                            "operation" : 'Int8Sum'}
                model[layer][search_candidate]['op_4']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_4']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_4']['input_shape'] == x.get_dict_record()['input_shapes']]
                
        elif g == 2 :
            # Conv_Relu_0 (pointwise - group) 
            model[layer][search_candidate]['op_1'] = {"input_shape" : [input_shape[i], [input_shape[i][3], 1, 1, input_shape[i][3]//g], [input_shape[i][3]]],
                                                            "operation" : 'Int8ConvRelu',
                                                            "stride" : 1}
            model[layer][search_candidate]['op_1']["runtime_p50"] = [x.get_val('runtime_us_p50') 
                                                                            for x in ops if model[layer][search_candidate]['op_1']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_1']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_1']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])]

            # Int8ChannelShuffle for Group Convolution
            model[layer][search_candidate]['op_2'] = {"input_shape" : [input_shape[i]],
                                                            "operation" : 'Int8ChannelShuffle'}
            model[layer][search_candidate]['op_2']["runtime_p50"] = [x.get_val('runtime_us_p50') 
                                                                            for x in ops if model[layer][search_candidate]['op_2']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_2']['input_shape'] == x.get_dict_record()['input_shapes']]
            
            # Conv_Relu_1 (depthwise)
            model[layer][search_candidate]['op_3'] = {"input_shape" : [input_shape[i], [input_shape[i][3], k, k, 1], [input_shape[i][3]]],
                                                            "operation" : 'Int8ConvRelu',
                                                            "stride" : stride}
            model[layer][search_candidate]['op_3']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_3']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_3']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_3']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])]
           
            # Conv_Relu_2 (pointwise - group)
            model[layer][search_candidate]['op_4'] = {"input_shape" : [input_shape[i+1], [output_filter, 1, 1, input_shape[i][3]//g],[output_filter]],
                                                            "operation" : 'Int8Conv',
                                                            "stride" : 1}
            model[layer][search_candidate]['op_4']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_4']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_4']['input_shape'] == x.get_dict_record()['input_shapes'] and
                                                                           model[layer][search_candidate]['op_4']['stride'] == int(x.get_val('op_args')[-1].split(' ')[-1][0])] # and
                                                                          # 'exhaustive_search' not in x.get_dict_record()['op_args'][0]]
            
            # Int8Sum 
            if not i in non_inverse_residual_block:
                model[layer][search_candidate]['op_5'] = {"input_shape" : [input_shape[i+1], input_shape[i+1]],
                                                            "operation" : 'Int8Sum'}
                model[layer][search_candidate]['op_5']["runtime_p50"] = [x.get_val('runtime_us_p50') for x in ops if model[layer][search_candidate]['op_5']['operation'] == x.get_val('op_type') and
                       model[layer][search_candidate]['op_5']['input_shape'] == x.get_dict_record()['input_shapes']]
            
        else :
            print(search_candidate)


In [50]:
model

{'L1': {'k3_e6_g1': {'op_1': {'input_shape': [[1, 112, 112, 16],
     [96, 1, 1, 16],
     [96]],
    'operation': 'Int8ConvRelu',
    'stride': 1,
    'runtime_p50': [1411.0]},
   'op_2': {'input_shape': [[1, 112, 112, 96], [96, 3, 3, 1], [96]],
    'operation': 'Int8ConvRelu',
    'stride': 1,
    'runtime_p50': []},
   'op_3': {'input_shape': [[1, 112, 112, 96], [16, 1, 1, 96], [16]],
    'operation': 'Int8Conv',
    'stride': 1,
    'runtime_p50': []},
   'op_4': {'input_shape': [[1, 112, 112, 16], [1, 112, 112, 16]],
    'operation': 'Int8Sum',
    'runtime_p50': [98.0]}},
  'k3_e3_g1': {'op_1': {'input_shape': [[1, 112, 112, 16],
     [48, 1, 1, 16],
     [48]],
    'operation': 'Int8ConvRelu',
    'stride': 1,
    'runtime_p50': [754.0]},
   'op_2': {'input_shape': [[1, 112, 112, 48], [48, 3, 3, 1], [48]],
    'operation': 'Int8ConvRelu',
    'stride': 1,
    'runtime_p50': []},
   'op_3': {'input_shape': [[1, 112, 112, 48], [16, 1, 1, 48], [16]],
    'operation': 'Int8Conv',
  

## 3. check LUT

In [31]:
## check
count_o = 0
count_x = 0

count_o_dict = dict()
count_x_dict = dict()

temp_list = []
for key in model.keys():
    for key2 in model[key].keys():
        
        if key2 == 'skip' : continue
            
        temp = 0
        for key3 in model[key][key2].keys():
            op = model[key][key2][key3]['operation']
            
            if model[key][key2][key3]['runtime_p50'] == []:
                count_x += 1
                temp += 1
    
                if op in count_x_dict.keys():
                    count_x_dict[op] += 1
                else : 
                    count_x_dict[op] = 0
                
            else :
                count_o += 1 
                
                if op in count_o_dict.keys():
                    count_o_dict[op] += 1
                else : 
                    count_o_dict[op] = 0
                
                    
            print(model[key][key2][key3]['runtime_p50'])
            
        if temp != 0:
            temp_list.append(key +' - '+ key2)
            
print(temp_list)
print(len(temp_list))
            

[1411.0]
[]
[]
[98.0]
[754.0]
[]
[]
[98.0]
[254.0]
[198.0]
[251.0]
[98.0]
[]
[]
[198.0]
[]
[98.0]
[1411.0]
[]
[]
[98.0]
[754.0]
[]
[]
[98.0]
[254.0]
[]
[251.0]
[98.0]
[]
[]
[]
[]
[98.0]
[1411.0]
[350.0]
[290.0]
[754.0]
[198.0]
[187.0]
[254.0]
[]
[]
[]
[]
[]
[]
[1411.0]
[]
[290.0]
[754.0]
[]
[187.0]
[254.0]
[]
[]
[]
[]
[]
[]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[89.0]
[126.0]
[44.0, 46.0]
[]
[]
[89.0]
[]
[44.0, 46.0]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[291.0]
[126.0]
[44.0, 46.0]
[]
[]
[291.0]
[]
[44.0, 46.0]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[89.0]
[126.0]
[44.0, 46.0]
[]
[]
[89.0]
[]
[44.0, 46.0]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[291.0]
[126.0]
[44.0, 46.0]
[]
[]
[291.0]
[]
[44.0, 46.0]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[89.0]
[126.0]
[44.0, 46.0]
[]
[]
[89.0]
[]
[44.0, 46.0]
[642.0]
[]
[]
[44.0, 46.0]
[]
[]
[]
[44.0, 46.0]
[127.0]
[291.0]
[126.0]
[44.0, 46.0]
[]
[]
[291.0]


In [32]:
count_o, count_x

# (524, 176) : exhautisve search O (overlap occured)
# => (494, 206) : exhautisve search X

(494, 206)

In [33]:
count_o_dict

{'Int8ConvRelu': 243,
 'Int8Sum': 127,
 'Int8Conv': 105,
 'Int8ChannelShuffle': 15}

In [35]:
count_x_dict

{'Int8ConvRelu': 107, 'Int8Conv': 69, 'Int8ChannelShuffle': 27}

In [36]:
model.keys()

dict_keys(['L1', 'L2', 'L3', 'L4', 'L5', 'L6', 'L7', 'L8', 'L9', 'L10', 'L11', 'L12', 'L13', 'L14', 'L15', 'L16', 'L17', 'L18', 'L19', 'L20', 'L21', 'L22'])

In [39]:
model['L1']

{'k3_e6_g1': {'op_1': {'input_shape': [[1, 112, 112, 16],
    [96, 1, 1, 16],
    [96]],
   'operation': 'Int8ConvRelu',
   'stride': 1,
   'runtime_p50': [1411.0]},
  'op_2': {'input_shape': [[1, 112, 112, 96], [96, 3, 3, 1], [96]],
   'operation': 'Int8ConvRelu',
   'stride': 1,
   'runtime_p50': []},
  'op_3': {'input_shape': [[1, 112, 112, 96], [16, 1, 1, 96], [16]],
   'operation': 'Int8Conv',
   'stride': 1,
   'runtime_p50': []},
  'op_4': {'input_shape': [[1, 112, 112, 16], [1, 112, 112, 16]],
   'operation': 'Int8Sum',
   'runtime_p50': [98.0]}},
 'k3_e3_g1': {'op_1': {'input_shape': [[1, 112, 112, 16],
    [48, 1, 1, 16],
    [48]],
   'operation': 'Int8ConvRelu',
   'stride': 1,
   'runtime_p50': [754.0]},
  'op_2': {'input_shape': [[1, 112, 112, 48], [48, 3, 3, 1], [48]],
   'operation': 'Int8ConvRelu',
   'stride': 1,
   'runtime_p50': []},
  'op_3': {'input_shape': [[1, 112, 112, 48], [16, 1, 1, 48], [16]],
   'operation': 'Int8Conv',
   'stride': 1,
   'runtime_p50': []}